# Der Bibelcode

### Einführung


1997 hat ein Buch des Journalisten Michael Drosnin grosse Aufmerksamkeit erregt. Das Buch mit dem Namen *Bible code* postuliert, dass in der Bibel (genauer gesagt in der Tora) versteckte Botschaften codiert sind. Diese Botschaften sind durch equidistante Folgen von Zeichen codiert, wie im folgenden Bild gezeigt ist (Quelle: Wikipedia)
<figure style="align:center">
    <img src="images/bible-code.png" width=200px/>
</figure>

In dieser Fallstudie werden wir überprüfen, ob wir auch in anderen Werken solche Botschaften finden. Wir nutzen dazu das Buch Krieg und Frieden von Leo Tolstoy, welches frei auf [Projekt Gutenberg](https://www.gutenberg.org) verfügbar ist. 

### Aufgabe

Unser Ziel ist es in einem Text alle als *Bible code* versteckten Wörter zu finden welche in einem Wörterbuch vorkommen. Dabei ist der Text in einer einfachen `.txt`-Datei gegeben und das Wörterbuch als eine `.json`-Datei.

### Problem Analyse und Zerlegung

Wir wollen als erstes das ganze Problem in Teilprobleme zerlegen. Dabei sollen die Teilprobleme einfach zu implementieren sein und sich auch gut zur gesamten Lösung zusammensetzten lassen.

Überlegen Sie sich Teilprobleme und ergänzen Sie die Liste damit. Überlegen Sie sich auch, welche Teilprobleme Sie einfach als Funktion implementieren können:

- Einlesen des Textes aus einer .txt-Datei.
- Einlesen eines Wörterbuches aus einer .json-Datei.
- ...
- *Entfernen von Sondernzeichen aus einem Text (und umwandeln in Kleinbuchstaben).*
- *Extrahieren eines Wortes aus einem Text an einer Stelle mit gegebener Distanz zwischen den Buchstaben.*
- *Testen von allen Worten im Dictionary.*
- *Speichern einer Stelle wo ein Wort gefunden wurde.*
- *Ausgabe aller Stellen wo ein Wort gefunden wurde.*
- *Ausgabe wie oft ein Wort gefunden wurde.*

### Vorbereiten der Daten

Als Erstes wollen wir die Vorverarbeitung der Daten erstellen. Wir wollen unsere Chancen auf einen Treffer erhöhen indem wir von einem Text möglichst nur die Buchstaben behalten. Dazu schreiben wir uns eine erste Hilfsfunktion. Diese soll: 
- auf Basis eines übergebenen Textes einen neuen Text erstellen.
- möglichst nur Buchstaben behalten.
- die Buchstaben in Kleinbuchstaben umwandeln.
- Jedes Sonderzeichen durch ein Leerzeichen ersetzen. 

In [ ]:
def clean_text(text):
    s = "" 
    for c in text:
        if c in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ":
            s += c.lower()
        else: 
            s += " "
    return s

Da wir all unseren Code den wir schreiben immer auch testen sollen, rufen wir die Funktion mit einem einfachen Text auf. Wir schauen dann ob die Funktion das tut was wir erwarten.

In [ ]:
clean_text("Eine Frage? ein Ausrufezeichen! und ein Punkt.")

### Text laden
Als nächstes wollen wir das laden des Textes aus einem File implementieren. Auch dazu schreiben wir uns eine Funktion welche den Inhalt des Files `war-and-peace.txt` laden soll. Der Text soll dann gleich noch von Sonderzeichen befreit werden.

In [ ]:
def load_preprocessed_text():    
    f = open("war-and-peace.txt", 'r')
    text = f.read()
    return clean_text(text)

Testen wir doch auch diese Funktion gleich indem wir die ersten 100 Zeichen ausgeben.

In [ ]:
load_preprocessed_text()[0:100]

### Laden des Dictionaries

Das Dictionary liegt in einem speziellen Format, dem *json* format vor. Dies ist ein oft benutztes Datenformat zum Austausch von Daten. Das *json* format kann direkt in ein Dictionary umgewandelt werden. 

In [ ]:
def load_dictionary():
    import json
    f = open("dictionary.json", "r")
    dictionary = json.load(f)
    return dictionary

# Testen inklusive
dictionary = load_dictionary()
print("{} words loaded into the dictionary".format(len(dictionary)))

Probieren wir doch aus, ob wir Definitionen für einige Wichtige Wörter erhalten. 

In [ ]:
print("python: ", dictionary["python"])
print("")
print("code: ", dictionary["code"])

### Äquidistance Wörter fixer Länge

Wir schreiben uns nun eine Funktion, welche Wörter, bestehend aus äquidistanten Buchstabensequenzen, extrahiert. Von einer bestimmten Startposition ausgehend, sollen alle Worte gebildet werden, die länger als eine angegebene Mindestlänge sind und kleiner als eine angegebene Maximallänge.

Die Worte werden in einer Liste gespeichert, die an den Aufrufer zurückgegeben wird. 

In [ ]:
def equidistant_char_seq(text, start_position, distance, min_word_len=5, max_word_len=15):
    words = []
    word = "" 
    pos = start_position
    word_len = 0
    while word_len < max_word_len and pos < len(text):
        word = word + text[pos]
        pos = pos + distance
        if (word_len >= min_word_len - 1):
            words.append(word)
        word_len = word_len + 1
        
    return words

In [ ]:
equidistant_char_seq("paybtchcodn", 0, 2, min_word_len = 5, max_word_len=7)

Wir wollen uns die Information speichern, wo ein Wort innerhalb eines Texts gefunden wurde, und mit welcher Distanz dies extrahiert wurde. Dazu verwenden wir eine Klasse. 

In [ ]:
class WordOccurrence:
    def __init__(self, word, pos, distance):
        self.word = word
        self.pos = pos
        self.distance = distance
        
    
    def __repr__(self):
        return ("Word: \"{}\", Position: {}, Distance: {}".format(self.word, self.pos, self.distance))

Und nun können wir eine einfache Funktion schreiben, die für eine gegebene Distanz an jeder Stelle im Code die Worte extrahiert, und schaut ob die im Dictionary vorkommen (also ob es sich um richtige Worte handelt). Wir speichern jedes gefunden Wort in einer Liste. 

In [ ]:
def find_all_words(text, dictionary, distance):
    occurrences = []
    for start_position in range(0,len(text)):
        words = equidistant_char_seq(text, start_position, distance)
        for word in words:
            if word in dictionary:
                occurrences.append(WordOccurrence(word, start_position, distance))
    return occurrences

# Wir wissen was raus kommen sollte...
dictionary = load_dictionary()
#find_all_words(text, dictionary, 2)
find_all_words("apbyctdheofngsaanbbacckdde", dictionary, 2)

Und nun lassen wir das auf unserem Text "War and Peace" laufen. Da es sich hierbei um einen ziemlichen Wälzer handelt, kann dies einige Zeit dauert. Um es etwas effizienter machen, durchsuchen wir nur die ersten hunderttausend Zeichen.  

In [ ]:
text = load_preprocessed_text()
dictionary = load_dictionary()

found_words = find_all_words(text[0:100000], dictionary, 5)

Sie sehen, solche Worte finden wir zuhauf. 
Zum Schluss geben wir noch die Textstelle für jedes Wort aus:

In [ ]:
for word_occurence in found_words:
    pos = word_occurence.pos
    word = word_occurence.word
    dist = word_occurence.distance
    
    snippet = text[pos : word_occurence.pos + dist * len(word)]

    print("Word {} found in snippet {} (distance {}))".format(word, snippet, dist))